<a href="https://colab.research.google.com/github/mokymok/notebooks/blob/main/markovbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install asyncio
!pip install numpy
!pip install discord

     |████████████████████████████████| 102kB 5.9MB/s 


     |████████████████████████████████| 788kB 8.3MB/s 
     |████████████████████████████████| 1.3MB 40.6MB/s 
     |████████████████████████████████| 296kB 46.9MB/s 
     |████████████████████████████████| 143kB 48.2MB/s 


In [ ]:
AUTOGEN_SETTO    = '**Auto-generation rate set to {0}**'

import discord
from discord import Intents
import asyncio
from discord.ext import commands
import numpy as np
import random
import re
from collections import defaultdict

intents = Intents.all()
bot = commands.Bot(intents = intents, command_prefix = "!")
token = ""
loop = asyncio.get_event_loop()
print(loop)
self = discord.User
final_message = ""
channels = {}

@bot.event
async def on_ready():
  print("Logged on as", self.user)
  await bot.change_presence(
      activity = discord.Activity(
          type = discord.ActivityType.listening, 
          name = "You"))

@bot.command(pass_context=True, name='autorate')
async def autorate_cmd(ctx, *args):
    chan_info = channels[ctx.message.channel.id]
    autorate = args[0]
    try:
        autorate = int(autorate)
        if autorate < 0:
            await ctx.send(INVALID_ARG)
            return
        chan_info['autorate'] = autorate
        await ctx.send(AUTOGEN_SETTO.format(str(autorate)))

@bot.command(pass_context=True)
async def test(ctx):
    await ctx.send(final_message)

@bot.command(pass_context=True)
async def join(ctx):
  chan = ctx.author.voice.channel
  await chan.connect()

@bot.command(pass_context=True)
async def leave(ctx):
  await ctx.voice_client.disconnect()

@bot.listen("on_message")
async def train(ctx):
    text = ctx.content
    if ctx.author == self.User:
        return

    if text.startswith(bot.command_prefix):
        await bot.process_commands(ctx)
        return

    model_file = open("model.txt", "a")
    model_file.write(text)
    model_file.close()

    tokenized_text = [
        word
        for word in re.split("\W+", text)
        if word != ""
]

        # Create graph.
    markov_graph = defaultdict(lambda: defaultdict(int))

    last_word = tokenized_text[0].lower()
    for word in tokenized_text[1:]:
        word = word.lower()
        markov_graph[last_word][word] += 1
        last_word = word

    #     # Preview graph.
    # limit = 3
    # for first_word in ("the", "by", "who"):
    #     next_words = list(markov_graph[first_word].keys())[:limit]
    #     for next_word in next_words:
    #         print(first_word, next_word)

    if chan_id not in channels:
        chan_info = {
            'autorate':  4,    'total_msgs': 0,
            'next_gen_milestone': chan_info['autorate']
        }
        channels[chan_id] = chan_info
    
    chan_info = channels[chan_id]
    chan_info['total_msgs'] += 1

    def walk_graph(graph, distance=5, start_node=None):
        """Returns a list of words from a randomly weighted walk."""
        if distance <= 0:
            return []
  
            # If not given, pick a start node at random.
        if not start_node:
            start_node = random.choice(list(graph.keys()))
  
  
        weights = np.array(
            list(markov_graph[start_node].values()),
            dtype=np.float64)
            # Normalize word counts to sum to 1.
        weights /= weights.sum()

            # Pick a destination using weighted distribution.
        choices = list(markov_graph[start_node].keys())
        chosen_word = np.random.choice(choices, None, p=weights)
  
        return [chosen_word] + walk_graph(
            graph, distance=distance-1,
            start_node=chosen_word)

    for i in range(1):
        final_message = " ".join(walk_graph(markov_graph, distance=random.randint(5,250)))
        print(final_message)
    
    chan_id = int(channel.id)
    if channel_exists(chan_id) and chan_id not in channels:
        await load_channel(chan_id, channel)

    rate = chan_info['autorate']
    if rate > 0 and chan_info['total_msgs'] >= chan_info['next_gen_milestone']:
        # set next milestone
        qrate = rate / 4
        chan_info['next_gen_milestone'] = chan_info['total_msgs'] + random.randint(rate - qrate, rate + qrate)
        await ctx.send(final_message)

if loop.is_running() == False:
    bot.run(token)